In [ ]:
from google.colab import files
files.upload()

!rm -r ~/.kaggle
!rm -r ~/content
!mkdir ~/.kaggle
!mkdir ~/content
!mv ./kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d vencerlanz09/sea-animals-image-dataste

import zipfile
zip_ref = zipfile.ZipFile('sea-animals-image-dataste.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
import tensorflow as tf
import random
import numpy as np
import os
import shutil

In [ ]:
dataset_path = "/content/"
train_path = "/content/train"
val_path = "/content/val"
test_path = "/content/test"

if not os.path.exists(train_path):
    os.mkdir(train_path)

if not os.path.exists(val_path):
    os.mkdir(val_path)

if not os.path.exists(test_path):
    os.mkdir(test_path)

In [ ]:
IMG_SIZE=256
labels = ['Penguin', 'Starfish', 'Sea Rays', 'Crabs', 'Otter', 'Whale']
print(labels)
print(len(labels))

In [ ]:
for label in labels:
    label_full_path = os.path.join(dataset_path, label)

    images_path = os.listdir(label_full_path)

    images = [i for i in images_path if i.endswith(".jpg")]

    random.shuffle(images)
    split_size = 0.8
    val_size = 0.1
    train_len = int(len(images) * split_size)
    val_len = train_len + int(len(images) * val_size)
    train_images = images[:train_len]
    val_images = images[train_len:val_len]
    test_images = images[val_len:]

    label_train_path = os.path.join(train_path, label)
    label_val_path = os.path.join(val_path, label)
    label_test_path = os.path.join(test_path, label)

    if not os.path.exists(label_train_path):
        os.mkdir(label_train_path)

    if not os.path.exists(label_val_path):
        os.mkdir(label_val_path)

    if not os.path.exists(label_test_path):
        os.mkdir(label_test_path)

    for im in train_images:
        shutil.copy(os.path.join(label_full_path, im), label_train_path)

    for im in val_images:
        shutil.copy(os.path.join(label_full_path, im), label_val_path)

    for im in test_images:
        shutil.copy(os.path.join(label_full_path, im), label_test_path)

In [ ]:
batch_size = 32

datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,
        zoom_range=0.1,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_ds = datagen.flow_from_directory(
    train_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    classes=labels,
    batch_size=batch_size
)

val_ds = test_datagen.flow_from_directory(
    val_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    classes=labels,
    batch_size=batch_size
)

test_ds = test_datagen.flow_from_directory(
    test_path,
    target_size=(IMG_SIZE, IMG_SIZE),
    classes=labels,
    batch_size=batch_size
)